In [1]:
from pyspark.sql import SparkSession
import pandas as pd

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('homework') \
    .getOrCreate()

23/03/08 11:21:24 WARN Utils: Your hostname, wedivv-H110M-S2V resolves to a loopback address: 127.0.1.1; using 192.168.1.44 instead (on interface wlp5s0)
23/03/08 11:21:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/08 11:21:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version 
# Question 1 - version

'3.3.2'

In [5]:
df_hvfhw = spark.read.csv('./data/fhvhv_tripdata_2021-06.csv.gz', header=True, inferSchema=True)

In [6]:
df_hvfhw.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|         174|          18|      N|                B02764|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|          32|         254|      N|                B02764|
|              B02764|2021-06-01 00:27:01|2021-06-01 00:42:11|         240|         127|      N|                B02764|
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                B02764|
|              B02510|2021-06-01 00:45:42|2021-06-01 01:03:33|         144|         146|      N|                  null|
|              B02510|2021-06-01 00:18:1

In [7]:
df_hvfhw.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [8]:
df_hvfhw = df_hvfhw.repartition(12)

In [ ]:
df_hvfhw.write.parquet('data/hvfhw/')
# Question 2: avg size of .parquet = 24MB

In [10]:
df_pqhvfhw = spark.read.parquet('data/hvfhw/')

In [11]:
df_pqhvfhw.count()

14961892

In [12]:
df_pqhvfhw.createOrReplaceTempView('hvfhw_data')

In [13]:
# Question 3: taxi trips on June 15 = 452.470

spark.sql("""
SELECT
    count(*)
FROM
    hvfhw_data
WHERE
    pickup_datetime LIKE '2021-06-15%'
    
""").show()


+--------+
|count(1)|
+--------+
|  452470|
+--------+



In [14]:
# Question 4: Longest trip for each day = 66.88 Hours

spark.sql("""
SELECT
    dispatching_base_num,
    ROUND((UNIX_TIMESTAMP(dropoff_datetime) - UNIX_TIMESTAMP(pickup_datetime)) / 3600.0, 2) AS trip_duration_hours
FROM
    hvfhw_data
ORDER BY trip_duration_hours DESC
""").show()

+--------------------+-------------------+
|dispatching_base_num|trip_duration_hours|
+--------------------+-------------------+
|              B02872|              66.88|
|              B02765|              25.55|
|              B02879|              19.98|
|              B02800|              18.20|
|              B02682|              16.47|
|              B02869|              14.27|
|              B02877|              13.91|
|              B02765|              11.67|
|              B02617|              11.37|
|              B02880|              10.98|
|              B02866|              10.27|
|              B02882|               9.97|
|              B02510|               9.97|
|              B02510|               9.64|
|              B02510|               9.62|
|              B02510|               9.48|
|              B02800|               9.47|
|              B02510|               9.40|
|              B02510|               9.39|
|              B02764|               9.38|
+----------

In [15]:
df_pqhvfhw.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [16]:
df_pqhvfhw = df_pqhvfhw.select("dispatching_base_num", "pickup_datetime", "PULocationID")


In [17]:
df_pqhvfhw = df_pqhvfhw.withColumnRenamed("pickup_datetime", "hour")


In [18]:
df_pqhvfhw.show()

+--------------------+-------------------+------------+
|dispatching_base_num|               hour|PULocationID|
+--------------------+-------------------+------------+
|              B02510|2021-06-04 08:13:36|          91|
|              B02875|2021-06-25 01:52:01|         132|
|              B02875|2021-06-25 03:34:13|          75|
|              B02866|2021-06-05 09:21:57|         233|
|              B02876|2021-06-01 01:26:49|         182|
|              B02883|2021-06-10 19:26:57|         140|
|              B02510|2021-06-10 09:28:54|         255|
|              B02510|2021-06-27 22:56:10|         198|
|              B02883|2021-06-17 09:09:16|          72|
|              B02510|2021-06-24 23:36:11|          79|
|              B02800|2021-06-03 02:26:01|         188|
|              B02876|2021-06-14 14:35:51|          45|
|              B02876|2021-06-12 17:04:35|          50|
|              B02882|2021-06-27 05:25:16|         198|
|              B02871|2021-06-27 13:50:47|      

In [19]:
zone = spark.read.csv('./data/taxi_zone_lookup.csv', header=True, inferSchema=True)

In [20]:
zone.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [ ]:
joined_df_pyspark = df_pqhvfhw.join(zone, df_pqhvfhw.PULocationID == zone.LocationID, "left_outer")


In [ ]:
joined_df_pyspark.show()

+--------------------+-------------------+------------+----------+---------+--------------------+------------+
|dispatching_base_num|               hour|PULocationID|LocationID|  Borough|                Zone|service_zone|
+--------------------+-------------------+------------+----------+---------+--------------------+------------+
|              B02510|2021-06-04 08:13:36|          91|        91| Brooklyn|           Flatlands|   Boro Zone|
|              B02875|2021-06-25 01:52:01|         132|       132|   Queens|         JFK Airport|    Airports|
|              B02875|2021-06-25 03:34:13|          75|        75|Manhattan|   East Harlem South|   Boro Zone|
|              B02866|2021-06-05 09:21:57|         233|       233|Manhattan| UN/Turtle Bay South| Yellow Zone|
|              B02876|2021-06-01 01:26:49|         182|       182|    Bronx|         Parkchester|   Boro Zone|
|              B02883|2021-06-10 19:26:57|         140|       140|Manhattan|     Lenox Hill East| Yellow Zone|
|

In [ ]:
joined_df_pyspark.createOrReplaceTempView('joined_data')

In [ ]:
# Question 6: Most frequent pickup location zone = 231279 pickups on Crown Heights North|

spark.sql("""
SELECT
    COUNT(dispatching_base_num) as pickups,
    Zone
FROM
    joined_data
GROUP BY
    Zone
ORDER BY
    pickups DESC
""").show()

+-------+--------------------+
|pickups|                Zone|
+-------+--------------------+
| 231279| Crown Heights North|
| 221244|        East Village|
| 188867|         JFK Airport|
| 187929|      Bushwick South|
| 186780|       East New York|
| 164344|TriBeCa/Civic Center|
| 161596|   LaGuardia Airport|
| 158937|            Union Sq|
| 154698|        West Village|
| 152493|             Astoria|
| 151020|     Lower East Side|
| 147673|        East Chelsea|
| 146402|Central Harlem North|
| 143683|Williamsburg (Nor...|
| 143594|          Park Slope|
| 141427|  Stuyvesant Heights|
| 139611|        Clinton East|
| 139431|West Chelsea/Huds...|
| 138428|             Bedford|
| 137879|         Murray Hill|
+-------+--------------------+
only showing top 20 rows



In [21]:
zone.createOrReplaceTempView('zone_data')

In [34]:
spark.sql("""
SELECT *
FROM hvfhw_data
""").show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02510|2021-06-04 08:13:36|2021-06-04 08:47:58|          91|          65|      N|                  null|
|              B02875|2021-06-25 01:52:01|2021-06-25 02:14:51|         132|         255|      N|                B02875|
|              B02875|2021-06-25 03:34:13|2021-06-25 03:42:22|          75|         263|      N|                B02875|
|              B02866|2021-06-05 09:21:57|2021-06-05 09:35:38|         233|         186|      N|                B02866|
|              B02876|2021-06-01 01:26:49|2021-06-01 01:33:02|         182|         212|      N|                B02876|
|              B02883|2021-06-10 19:26:5

In [36]:
spark.sql("""
SELECT *
FROM zone_data
""").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [77]:
spark.sql("""

SELECT
    z.Zone,
    COUNT(*) AS num_pickups
FROM
    hvfhw_data h
    LEFT JOIN zone_data z ON h.PULocationID = z.LocationID
GROUP BY
    z.Zone
ORDER BY
    num_pickups DESC
LIMIT 1

""").show()

+-------------------+-----------+
|               Zone|num_pickups|
+-------------------+-----------+
|Crown Heights North|     231279|
+-------------------+-----------+

